<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spark-MLlib-Tuning" data-toc-modified-id="Spark-MLlib-Tuning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-tuning.html" target="_blank">Spark MLlib Tuning</a></a></span></li><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="https://github.com/hyperopt/hyperopt" target="_blank">Hyperopt</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/" target="_blank">XGBoost Tuning</a></a></span><ul class="toc-item"><li><span><a href="#Objective-function" data-toc-modified-id="Objective-function-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Objective function</a></span></li><li><span><a href="#Tune-number-of-trees" data-toc-modified-id="Tune-number-of-trees-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Tune number of trees</a></span></li><li><span><a href="#Tune-tree-specific-parameters" data-toc-modified-id="Tune-tree-specific-parameters-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Tune tree-specific parameters</a></span><ul class="toc-item"><li><span><a href="#Tune-max_depth,-min_child_weight" data-toc-modified-id="Tune-max_depth,-min_child_weight-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Tune max_depth, min_child_weight</a></span></li><li><span><a href="#Tune-gamma" data-toc-modified-id="Tune-gamma-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Tune gamma</a></span></li><li><span><a href="#Tune-subsample,-colsample_bytree" data-toc-modified-id="Tune-subsample,-colsample_bytree-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Tune subsample, colsample_bytree</a></span></li></ul></li><li><span><a href="#Tune-regularization-parameters" data-toc-modified-id="Tune-regularization-parameters-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Tune regularization parameters</a></span></li><li><span><a href="#Lower-the-learning-rate-and-decide-the-optimal-parameters" data-toc-modified-id="Lower-the-learning-rate-and-decide-the-optimal-parameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lower the learning rate and decide the optimal parameters</a></span></li></ul></li><li><span><a href="#LogisticRegression-Tuning" data-toc-modified-id="LogisticRegression-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression Tuning</a></span></li><li><span><a href="#Optional-MongoTrials" data-toc-modified-id="Optional-MongoTrials-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optional <a href="https://hyperopt.github.io/hyperopt/scaleout/mongodb/" target="_blank">MongoTrials</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>XGBoost Tuning</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

COMMON_PATH = '/workspace/common'

sys.path.append(os.path.join(COMMON_PATH, 'utils'))

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars {common}/xgboost4j-spark-0.72.jar,{common}/xgboost4j-0.72.jar
--py-files {common}/sparkxgb.zip pyspark-shell
""".format(common=COMMON_PATH).replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, get_ate
from processing import split_by_col

from sparkxgb.xgboost import *

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

**Remark** Необязательно использовать половину датасета и всего две категориальные переменные. Можно использовать больше данных, если вам позволяет ваша конфигурация

In [4]:
df = df.sample(False, 0.5)

In [5]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [6]:
df = df.fillna(0, subset=num_columns)

In [7]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load('/workspace/data/criteo_models/mllib_practice_model')

In [8]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

1831350

In [9]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

# [Spark MLlib Tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

У имеющегося в Spark'e метода HPO есть два существенных недостатка, которые делают его мало пригодным в контексте нашей задачи:

1. `ParamGridBuilder` - поиск по сетке
2. `TrainValidationSplit` - делит данные случайнм образом

# [Hyperopt](https://github.com/hyperopt/hyperopt)

Установим `hyperopt`

In [10]:
!pip3.5 install hyperopt

## [XGBoost Tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

> [Notes on Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

### Objective function

In [11]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import scipy.stats as st


def objective(space):
    estimator = XGBoostEstimator(**space)
    print('SPACE:', estimator._input_kwargs_processed())
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [12]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 1,
    'silent': 0,
    'nworkers': 1
}

Fix baseline parameters and train baseline model

In [13]:
CONTROL_NAME = 'xgb baseline'

baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

In [14]:
baseline_rocauc = rocauc(baseline_model, val_df, probabilities_col='probabilities')
baseline_rocauc

0.7241966892958183

In [15]:
all_metrics = {}

In [16]:
baseline_test_metrics = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

all_metrics[CONTROL_NAME] = baseline_test_metrics

### Tune number of trees

> Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate.

In [45]:
%%time

num_round_choice = [10, 20, 40, 100]
eta_choice = [0.5, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'gamma': 3.0, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.1, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 40, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5092394095053826, ROC-AUC: 0.7267305995939046
SPACE:                                                                           
{'gamma': 3.0, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.5, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 40, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5037994502941576, ROC-AUC: 0.7340043146850984                        
SPACE:                                                                           
{'gamma': 3.0, 'nthread

LOG-LOSS: 0.5172744485282739, ROC-AUC: 0.7208762944896147                       
SPACE:                                                                          
{'gamma': 3.0, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.501821839869757, ROC-AUC: 0.7370586785416808                        
SPACE:                                                                           
{'gamma': 3.0, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.2, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.503361131047961, ROC-AUC: 0.735061363171860

In [46]:
best

{'eta': 4, 'num_round': 3}

Обратите внимание на то, что в случае с `hp.choice` в переменной `best` хранится не конкретное значение гиперпараметра, а его индекс из списка, например, `num_round_choice`

In [47]:
eta = eta_choice[best['eta']]  # change me!
num_round = num_round_choice[best['num_round']]  # change me!

In [48]:
current_params = baseline_params
current_params['eta'] = eta
current_params['num_round'] = num_round

In [49]:
model = XGBoostEstimator(**{**static_params, **current_params}).fit(train_df)

In [50]:
CURRENT_NAME = "opt-model-1"

current_test_metrics = {
    'logloss': logloss(model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(model, test_df, probabilities_col='probabilities')
}

all_metrics[CURRENT_NAME] = current_test_metrics

In [51]:
all_metrics

{'opt-model-1': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588844},
 'opt-model-2': {'logloss': 0.5042451682962059, 'rocauc': 0.7365672720796711},
 'opt-model-3': {'logloss': 0.5042492745165619, 'rocauc': 0.7366437850705707},
 'opt-model-4': {'logloss': 0.5040673148590921, 'rocauc': 0.7367384318514827},
 'opt-model-5': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588867},
 'xgb baseline': {'logloss': 0.513826416019165, 'rocauc': 0.7240490284428945}}

### Tune tree-specific parameters

> Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.

#### Tune max_depth, min_child_weight

In [52]:
%%time

max_depth_choice = [2, 4, 6]
min_child_weight_choice = [10., 50., 100., 500.]

space = {
    # Optimize
    'max_depth': hp.choice('max_depth', max_depth_choice),
    'min_child_weight': hp.choice('min_child_weight', min_child_weight_choice),
    
    # Fixed    
    'num_round': current_params['num_round'],
    'eta': current_params['eta'],
    'subsample': current_params['subsample'],
    'gamma': current_params['gamma'],
    'colsample_bytree': current_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'gamma': 3.0, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 500.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5031009494559164, ROC-AUC: 0.7353114406860606
SPACE:                                                                             
{'gamma': 3.0, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 500.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 4, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5049244223559074, ROC-AUC: 0.732759323206161                           
SPACE:                                                                             
{'gamma': 3.0, 

LOG-LOSS: 0.5105727205398666, ROC-AUC: 0.7250413813003515                        
SPACE:                                                                             
{'gamma': 3.0, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 10.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 4, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5043250850657147, ROC-AUC: 0.7336738775685587                          
SPACE:                                                                             
{'gamma': 3.0, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.501821839869757, ROC-AUC: 0.7370586

In [53]:
current_params['max_depth'] = max_depth_choice[best['max_depth']]
current_params['min_child_weight'] = min_child_weight_choice[best['min_child_weight']]

In [54]:
model = XGBoostEstimator(**{**static_params, **current_params}).fit(train_df)

CURRENT_NAME = "opt-model-2"

current_test_metrics = {
    'logloss': logloss(model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(model, test_df, probabilities_col='probabilities')
}

all_metrics[CURRENT_NAME] = current_test_metrics

In [55]:
all_metrics

{'opt-model-1': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588844},
 'opt-model-2': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588866},
 'opt-model-3': {'logloss': 0.5042492745165619, 'rocauc': 0.7366437850705707},
 'opt-model-4': {'logloss': 0.5040673148590921, 'rocauc': 0.7367384318514827},
 'opt-model-5': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588867},
 'xgb baseline': {'logloss': 0.513826416019165, 'rocauc': 0.7240490284428945}}

#### Tune gamma

In [56]:
%%time

gamma_choice = [.1, .5, 1., 3., 10., 50., 100.]

space = {
    # Optimize
    'gamma': hp.choice('gamma', gamma_choice),
    
    # Fixed
    'num_round': current_params['num_round'],
    'eta': current_params['eta'],
    'max_depth': current_params['max_depth'],
    'min_child_weight': current_params['min_child_weight'],
    'subsample': current_params['subsample'],
    'colsample_bytree': current_params['colsample_bytree'],

    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'gamma': 50.0, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.501821839869757, ROC-AUC: 0.7370586785416824                           
SPACE:                                                                             
{'gamma': 0.1, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5017008659080565, ROC-AUC: 0.7371983697748545                         
SPACE:                                                                   

In [57]:
current_params['gamma'] = gamma_choice[best['gamma']]

In [58]:
model = XGBoostEstimator(**{**static_params, **current_params}).fit(train_df)

CURRENT_NAME = "opt-model-3"

current_test_metrics = {
    'logloss': logloss(model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(model, test_df, probabilities_col='probabilities')
}

all_metrics[CURRENT_NAME] = current_test_metrics

In [59]:
all_metrics

{'opt-model-1': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588844},
 'opt-model-2': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588866},
 'opt-model-3': {'logloss': 0.5040476600477894, 'rocauc': 0.7368304724249373},
 'opt-model-4': {'logloss': 0.5040673148590921, 'rocauc': 0.7367384318514827},
 'opt-model-5': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588867},
 'xgb baseline': {'logloss': 0.513826416019165, 'rocauc': 0.7240490284428945}}

#### Tune subsample, colsample_bytree

In [60]:
%%time

subsample_choice = [0.2, 0.5, 0.7, 1.0]
colsample_bytree_choice = [0.3, 0.5, 0.7, 1.0]

space = {
    # Optimize
    'subsample': hp.choice('subsample', subsample_choice),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bytree_choice),
    
    # Fixed
    'num_round': current_params['num_round'],
    'eta': current_params['eta'],
    'max_depth': current_params['max_depth'],
    'min_child_weight': current_params['min_child_weight'],
    'gamma': current_params['gamma'],

    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'gamma': 0.5, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 0.7, 'min_child_weight': 100.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 0.7, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5021379949617328, ROC-AUC: 0.7366509276553932
SPACE:                                                                             
{'gamma': 0.5, 'nthread': 1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 6, 'colsample_bytree': 0.3, 'predictionCol': 'prediction', 'num_round': 100, 'nworkers': 1, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5026733982549004, ROC-AUC: 0.7358527976208282                          
SPACE:                                                                             
{'gamma': 0.5, 

In [61]:
current_params['subsample'] = subsample_choice[best['subsample']]
current_params['colsample_bytree'] = colsample_bytree_choice[best['colsample_bytree']]

In [62]:
model = XGBoostEstimator(**{**static_params, **current_params}).fit(train_df)

CURRENT_NAME = "opt-model-4"

current_test_metrics = {
    'logloss': logloss(model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(model, test_df, probabilities_col='probabilities')
}

all_metrics[CURRENT_NAME] = current_test_metrics

In [63]:
all_metrics

{'opt-model-1': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588844},
 'opt-model-2': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588866},
 'opt-model-3': {'logloss': 0.5040476600477894, 'rocauc': 0.7368304724249373},
 'opt-model-4': {'logloss': 0.5040476600477894, 'rocauc': 0.7368304724249363},
 'opt-model-5': {'logloss': 0.5038050594301458, 'rocauc': 0.7371370245588867},
 'xgb baseline': {'logloss': 0.513826416019165, 'rocauc': 0.7240490284428945}}

### Tune regularization parameters

> Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.

In [ ]:
%%time

reg_alpha_choice = [0., 0.1, 0.5, 1.]
reg_lambda_choice = [0.1, 0.5, 1., 2.]

space = {
    # Optimize
    'alpha': hp.choice('alpha', reg_alpha_choice),
    'reg_lambda': hp.choice('reg_lambda', reg_lambda_choice),
    
    # Fixed
    'num_round': current_params['num_round'],
    'eta': current_params['eta'],
    'max_depth': current_params['max_depth'],
    'min_child_weight': current_params['min_child_weight'],
    'gamma': current_params['gamma'],
    'subsample': current_params['subsample'],
    'colsample_bytree': current_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)
best

SPACE:                                                
{'gamma': 0.5, 'alpha': 0.1, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 6, 'nworkers': 1, 'lambda': 0.1, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'nthread': 1, 'num_round': 100, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.502049101097838, ROC-AUC: 0.7367001365217775
SPACE:                                                                            
{'gamma': 0.5, 'alpha': 1.0, 'objective': 'binary:logistic', 'silent': 0, 'subsample': 1.0, 'min_child_weight': 100.0, 'eta': 0.3, 'featuresCol': 'features', 'max_depth': 6, 'nworkers': 1, 'lambda': 1.0, 'colsample_bytree': 1.0, 'predictionCol': 'prediction', 'nthread': 1, 'num_round': 100, 'labelCol': 'label', 'eval_metric': 'logloss'}
LOG-LOSS: 0.5019636609775618, ROC-AUC: 0.7368997910797406                         
SPACE:                                      

In [ ]:
current_params['alpha'] = reg_alpha_choice[best['alpha']]
current_params['reg_lambda'] = reg_lambda_choice[best['reg_lambda']]

In [ ]:
model = XGBoostEstimator(**{**static_params, **current_params}).fit(train_df)

CURRENT_NAME = "opt-model-5"

current_test_metrics = {
    'logloss': logloss(model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(model, test_df, probabilities_col='probabilities')
}

all_metrics[CURRENT_NAME] = current_test_metrics

In [ ]:
all_metrics

### Lower the learning rate and decide the optimal parameters

In [ ]:
# Если я верно понимаю, что просится в этом задании -- это займет вечность

%%time

reg_alpha_choice = [0., 0.1, 0.5, 1.]
reg_lambda_choice = [0.1, 0.5, 1., 2.]
subsample_choice = [0.2, 0.5, 0.7, 1.0]
colsample_bytree_choice = [0.3, 0.5, 0.7, 1.0]
gamma_choice = [.1, .5, 1., 3., 10., 50., 100.]
num_round_choice = [10, 20, 40, 100]
eta_choice = [0.5, 0.10, 0.15, 0.20, 0.30]
max_depth_choice = [2, 4, 6]
min_child_weight_choice = [10., 50., 100., 500.]
eta = 0.05

space = {
    # Optimize
    'alpha': hp.choice('alpha', reg_alpha_choice),
    'reg_lambda': hp.choice('reg_lambda', reg_lambda_choice),
    'alpha': hp.choice('alpha', subsample_choice),
    'reg_lambda': hp.choice('reg_lambda', colsample_bytree_choice),
    'alpha': hp.choice('alpha', gamma_choice),
    'reg_lambda': hp.choice('reg_lambda', max_depth_choice),
    'reg_lambda': hp.choice('reg_lambda', reg_lambda_choice),
    
    # Fixed
    'num_round': current_params['num_round'],
    'eta': current_params['eta'],
    'max_depth': current_params['max_depth'],
    'min_child_weight': current_params['min_child_weight'],
    'gamma': current_params['gamma'],
    'subsample': current_params['subsample'],
    'colsample_bytree': current_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)
best

---
## LogisticRegression Tuning

Подберем гиперпараметры для логрега из предыдущих практик

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

---
## Optional [MongoTrials](https://hyperopt.github.io/hyperopt/scaleout/mongodb/)

> For parallel search, hyperopt includes a MongoTrials implementation that supports asynchronous updates.

**TLDR** Преимущества использования `MongoTrials`:
* `MongoTrials` позволяет параллельно запускать несколько вычислений целевой функции
* Динамический уровень параллелизма - можно добавлять/удалять воркеров, которые вычисляют целевую функцию
* Все результаты сохраняются в БД - история запусков никуда не потеряется

*За выполнение данного задания можно получить дополнительно +0.4 к итоговому баллу*

### XGBoost Tuning

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

# Results

Подведем итоги.

Обучите модели с найденными (оптимальными) гиперпараметрами и сделайте справнение на отложенной выборке

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

Итоговая таблица

In [ ]:
get_ate(all_metrics, CONTROL_NAME)